# Preprocesamiento

In [1]:
!pip install geopandas shapely rasterio Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 948.0 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 4.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 6.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 4.8 MB/s eta 0:00:0000:0100:01m


In [2]:
import numpy as np
import rasterio
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from pyproj import Proj, Transformer

## Generar dataframe base

In [3]:
ruta_denue_csv = "input_data/denue.csv"
try:
    df = pd.read_csv(ruta_denue_csv, encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(ruta_denue_csv, encoding="latin1")
df.head()

/tmp/ipykernel_22/2136972834.py:5: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_denue_csv, encoding="latin1")


,id,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
0,6862345,01001115119000031001000000U2,AGROPECUARIA CHARCOS DE QUEZADA,AGROPECUARIA CHARCOS DE QUEZADA SOCIEDAD DE PR...,115119,Otros servicios relacionados con la agricultura,6 a 10 personas,AVENIDA,UNIVERSIDAD,CALLE,...,Aguascalientes,0233,9,4492853716,NaN,NaN,Fijo,21.906992,-102.309807,2019-04
1,6905212,01001115119000041000000000U1,AGROPECUARIO LOS ALAMITOS,NaN,115119,Otros servicios relacionados con la agricultura,0 a 5 personas,CALLE,J. ENCARNACION PAYEN,CALLE,...,Aguascalientes,0322,18,4499960472,NaN,NaN,Fijo,21.889926,-102.314009,2019-11
2,6166248,01007112512000012001000000U8,CENTRO ACUÍCOLA PABELLON DE HIDALGO,SAGARPA,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,CALLE,NINGUNO,CALLE,...,SEMARNAT [Centro Acuícola],0078,800,NaN,NaN,WWW.GOB.MX,Fijo,22.167778,-102.345556,2010-07
3,6166233,01010112512000011000000000U9,G. ACUÍCOLA SANTA ROSA,NaN,112512,"Piscicultura y otra acuicultura, excepto camar...",6 a 10 personas,CARRETERA,SECTOR DE PRODUCCION N0 1 TERREMOTO EL LLANO A...,CARRETERA,...,Santa Rosa [Granja Acuícola],0069,800,NaN,NaN,NaN,Fijo,21.956298,-101.997312,2014-12
4,6166234,01003112512000031001000000U7,GRANJA ACUICOLA ARROYO DE LOS CABALLOS,NaN,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,PROLONGACION,CUAUHTUMOC,CARRETERA,...,Arroyo Ojocalientillo,0100,2,4492741517,MARTINSAU2720@HOTMAIL.COM,NaN,Fijo,21.837255,-102.710931,2014-12


In [4]:
df_limpio = df[["codigo_act", "latitud", "longitud"]].copy()
df_limpio["codigo_act"] = df_limpio["codigo_act"].astype(str).str[:2]

In [5]:
print("Valores nulos por columna:")
df_limpio.isnull().sum()

Valores nulos por columna:


codigo_act    0
latitud       0
longitud      0
dtype: int64

In [6]:
df_limpio.dtypes

codigo_act     object
latitud       float64
longitud      float64
dtype: object

In [7]:
ruta_landsat_tiff = "input_data/imagen.tif"

In [8]:
with rasterio.open(ruta_landsat_tiff) as src:
    srs_raster = src.crs.to_proj4()

In [9]:
# Crear columna "geometry" utilizando latitud y longitud
df_limpio["geometry"] = [Point(xy) for xy in zip(df_limpio["longitud"], df_limpio["latitud"])]

# Convertir el DataFrame de pandas en un GeoDataFrame de GeoPandas
gdf = gpd.GeoDataFrame(df_limpio, geometry="geometry")
srs_gdf = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
transformador = Transformer.from_proj(Proj(srs_gdf), Proj(srs_raster))
gdf["geometry2"] = gdf["geometry"].apply(lambda geom: Point(transformador.transform(geom.x, geom.y)))
gdf.head()

,codigo_act,latitud,longitud,geometry,geometry2
0,11,21.906992,-102.309807,POINT (-102.30981 21.90699),POINT (2468147.992 1094190.873)
1,11,21.889926,-102.314009,POINT (-102.31401 21.88993),POINT (2467711.796 1092292.619)
2,11,22.167778,-102.345556,POINT (-102.34556 22.16778),POINT (2464542.021 1123223.156)
3,11,21.956298,-101.997312,POINT (-101.99731 21.95630),POINT (2500276.264 1099643.835)
4,11,21.837255,-102.710931,POINT (-102.71093 21.83725),POINT (2426869.540 1086576.086)


In [10]:
# Abre el archivo GeoTIFF con rasterio
with rasterio.open(ruta_landsat_tiff) as src:
    # Itera sobre los puntos del GeoDataFrame y obtiene los valores de los pixeles correspondientes
    for index, row in gdf.iterrows():
        gdf.loc[index, ["col_pixel", "row_pixel"]] = src.index(row["geometry2"].x, row["geometry2"].y)


In [11]:
gdf

,codigo_act,latitud,longitud,geometry,geometry2,col_pixel,row_pixel
0,11,21.906992,-102.309807,POINT (-102.30981 21.90699),POINT (2468147.992 1094190.873),3526.0,2271.0
1,11,21.889926,-102.314009,POINT (-102.31401 21.88993),POINT (2467711.796 1092292.619),3590.0,2257.0
2,11,22.167778,-102.345556,POINT (-102.34556 22.16778),POINT (2464542.021 1123223.156),2559.0,2151.0
3,11,21.956298,-101.997312,POINT (-101.99731 21.95630),POINT (2500276.264 1099643.835),3345.0,3342.0
4,11,21.837255,-102.710931,POINT (-102.71093 21.83725),POINT (2426869.540 1086576.086),3780.0,895.0
...,...,...,...,...,...,...,...
61641,93,21.875367,-102.279581,POINT (-102.27958 21.87537),POINT (2471248.754 1090665.178),3644.0,2374.0
61642,93,21.865938,-102.289821,POINT (-102.28982 21.86594),POINT (2470193.605 1089617.948),3679.0,2339.0
61643,93,21.919051,-101.959755,POINT (-101.95976 21.91905),POINT (2504137.271 1095499.347),3483.0,3471.0
61644,93,21.893411,-102.323357,POINT (-102.32336 21.89341),POINT (2466751.468 1092682.561),3577.0,2225.0


In [12]:
gdf.shape

(61646, 7)

## Obtener recortes de 5, 3 y 1 pixeles
### Fast

In [13]:
_1p = np.empty([gdf.shape[0], 1,1,6])
_1p.shape

(61646, 1, 1, 6)

In [14]:
_3p = np.empty([gdf.shape[0], 3,3,6])
_5p = np.empty([gdf.shape[0], 5,5,6])

In [15]:
with rasterio.open(ruta_landsat_tiff) as src:
    for index, gdf_row in gdf.iterrows():
        row, col = gdf_row.row_pixel, gdf_row.col_pixel
        # 5 Pixels
        for sub_row in range(0, 5):
            for sub_col in range(0, 5):
                _5p[index][sub_row][sub_col] = src.read(
                    window=((row + (sub_row -2), row + (sub_row -1)),
                            (col + (sub_col -2), col + (sub_col -1)))
                ).reshape(_5p.shape[-1:])

In [16]:
_3p = _5p[:,1:4,1:4,:]

In [17]:
_1p = _3p[:,1,1,:]

### Slow

In [ ]:
with rasterio.open(ruta_landsat_tiff) as src:
    for index, gdf_row in gdf.iterrows():

        row, col = gdf_row.row_pixel, gdf_row.col_pixel
        _1p[index] = src.read(
            window=((row, row+1),(col, col+1))
        ).reshape(_1p.shape[1:])

        # 3 Pixels
        for sub_row in range(0, 3):
            for sub_col in range(0, 3):
                _3p[index][sub_row][sub_col] = src.read(
                    window=((row + (sub_row -1), row + sub_row),
                            (col + (sub_col -1), col + sub_col ))
                ).reshape(_3p.shape[-1:])


        # 5 Pixels
        for sub_row in range(0, 5):
            for sub_col in range(0, 5):
                _5p[index][sub_row][sub_col] = src.read(
                    window=((row + (sub_row -2), row + (sub_row -1)),
                            (col + (sub_col -2), col + (sub_col -1)))
                ).reshape(_5p.shape[-1:])

## Preparar labels

In [18]:
all_labels = gdf["codigo_act"].astype(int)

In [19]:
all_labels

0        11
1        11
2        11
3        11
4        11
         ..
61641    93
61642    93
61643    93
61644    93
61645    93
Name: codigo_act, Length: 61646, dtype: int64

In [20]:
all_labels.dtypes

dtype('int64')

## Generar modelo

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

2023-12-11 06:13:34.571286: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 06:13:34.594853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 06:13:34.594882: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 06:13:34.595594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 06:13:34.600469: I tensorflow/core/platform/cpu_feature_guar

In [22]:
# Crear el modelo de Deep Learning
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(3, 3, 6)))
#model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(all_labels), activation='softmax'))

# Compilar el modelo
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(_3p, all_labels, epochs=10, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(_3p, all_labels)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/10


2023-12-11 06:14:32.765711: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: UNKNOWN ERROR (34)


1542/1542 [==============================] - 133s 86ms/step - loss: 5.2179 - accuracy: 0.4074 - val_loss: 24.2121 - val_accuracy: 2.4331e-04
Epoch 2/10
1542/1542 [==============================] - 135s 87ms/step - loss: 2.0790 - accuracy: 0.4608 - val_loss: 37.4879 - val_accuracy: 0.0000e+00
Epoch 3/10
1542/1542 [==============================] - 126s 82ms/step - loss: 2.0667 - accuracy: 0.4619 - val_loss: 26.9447 - val_accuracy: 0.0000e+00
Epoch 4/10
1542/1542 [==============================] - 133s 87ms/step - loss: 2.2207 - accuracy: 0.4578 - val_loss: 33.4966 - val_accuracy: 0.0000e+00
Epoch 5/10
1542/1542 [==============================] - 133s 86ms/step - loss: 2.0297 - accuracy: 0.4624 - val_loss: 33.7139 - val_accuracy: 0.0000e+00
Epoch 6/10
1542/1542 [==============================] - 133s 86ms/step - loss: 2.0310 - accuracy: 0.4624 - val_loss: 29.6375 - val_accuracy: 0.0000e+00
Epoch 7/10
1542/1542 [==============================] - 133s 86ms/step - loss: 2.0511 - accuracy: 0

## Guardar modelo

In [23]:
# Guardar modelo para realizar predicciones sin volver a entrenar
model.save('output_model/geo_recortes.keras')

# Pendientes (Modelo) (Andrea, Andres)
- [ ] Ajustar modelo (parametros e hiperparametros)
- [ ] Train, test, validation splits
- [ ] Normalizar datos
- [ ] Generar gráficas de resultado de entrenamiento
- [ ] Integrar MLFlow? (comparar con recorte de 5 pixeles y 1 pixel)
- [ ] Verficar que calculo de recortes "rápido" es correcto
- [ ] Quitar números mágicos de este código


# Pendientes Api (flask)
- [ ] Reutilizar código para transformar lat, lon en recorte (Mitsiu)
- [ ] Cargar modelo (Mitsiu)
<code>
new_model = keras.models.load_model('geo_recortes.keras')
new_model.predict(new_data_from_flutter)
</code>
- [ ] Validaciónes (lat,lon dentro imágen )
- [ ] Integrar "SCIAN_2023124_141359987.xlsx" (ref notebook de Andrea) para mostrar labels entendibles

# Pendientes Front (flutter)
- [ ] Definir método (http) y parametros de comunicación con api y realizar petición - GET
- [ ] Mostrar resultados de predicción